In [1]:
## This notebook only runs with Colab Pro with High RAM and GPU

In [ ]:
!nvidia-smi

Sat Dec  2 15:10:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Dec  2 15:22:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
# Install the libraries

In [ ]:
!pip install langchain
!pip install sentence-transformers
!pip install chromadb
!pip install pypdf
!pip install auto-gptq
!pip install optimum
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=0d1c90b80829e4546157d270219aa85a1ca59d49754c1890447291b9cbffc252
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.4/502.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.9 MB/s 

In [ ]:
# Import the libraries

In [ ]:
import torch
import gradio as gr
import chromadb
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA #, ConversationalRetrievalChain
from langchain import PromptTemplate, HuggingFacePipeline, LLMChain
# from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, TextStreamer
import warnings
import time
warnings.simplefilter("ignore")

In [ ]:
import os

def create_directory(directory_path):
    # Check if the directory exists
    if not os.path.exists(directory_path):
        # If not, create it
        os.makedirs(directory_path)
        print(f"Directory '{directory_path}' created successfully.")
    else:
        print(f"Directory '{directory_path}' already exists.")

In [ ]:
# Example usage
directory_path = "/content/knowledge"
create_directory(directory_path)

Directory '/content/knowledge' created successfully.


In [ ]:
directory_path = "/content/vector_store"
create_directory(directory_path)

directory_path = "/content/model"
create_directory(directory_path)

Directory '/content/vector_store' created successfully.
Directory '/content/model' created successfully.


In [ ]:
# Upload Tesla and Meta earning reports to '/content/knowledge' folder

In [ ]:
class config:
  DATA_PATH = "/content/knowledge"
  DB_PATH = "/content/vector_store/db_chroma"
  DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
  EMBEDDINGS_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
  LLM_MODEL = "TheBloke/Llama-2-7B-chat-GPTQ"
  LOCAL_MODEL_DIR = "/content/model"
  MAX_NEW_TOKENS = 1024
  DEFAULT_SYSTEM_PROMPT = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
  Please ensure that your responses
  If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
  If you don't know the answer to a question, please don't share false information.
  """
  CUSTOM_PROMPT_TEMPLATE = """You are a helpful financial assistant. Use the following pieces of context to answer the user questions. Take a deep breath and think step by step before answering the question. Be precise in your answers whenever possible.
  If you can not answer a user question based on the provided context, just say that you don't know. Do not use any other information to answer the QUESTIONS
  """


In [ ]:
## 1. Create vector database with embeddings

# 1. Create vector database with embeddings

In [ ]:
#create vector database
def create_vector_db():
  loader = DirectoryLoader(config.DATA_PATH,
                          glob='*.pdf',
                          loader_cls=PyPDFLoader
                          )
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024,
                                                chunk_overlap=128
                                                )
  texts = text_splitter.split_documents(documents)
  print(texts)

  embeddings = HuggingFaceEmbeddings(model_name=config.EMBEDDINGS_MODEL,
                                    model_kwargs={'device': config.DEVICE}
                                    )
  db = Chroma.from_documents(texts, embeddings, persist_directory=config.DB_PATH)
  print("Vector database created with embeddings for the provided data")


In [ ]:
# %rm -rf "/content/vector_store"
# %rm -rf "/content/model


In [ ]:
create_vector_db()

[Document(page_content='Q3 2023 Update\n1', metadata={'source': '/content/knowledge/TSLA-Q3-2023-Update-3.pdf', 'page': 0}), Document(page_content='Highlights 03\nFinancial Summary 04\nOperational Summary 06\nVehicle Capacity 07\nCore Technology 08\nOther Highlights 09\nOutlook 10\nPhotos & Charts 11\nKey Metrics 16\nFinancial Statements 19\nAdditional Information 25', metadata={'source': '/content/knowledge/TSLA-Q3-2023-Update-3.pdf', 'page': 1}), Document(page_content='S U M M A R Y H I G H L I G H T S  \n(1) Excludes SBC (stock -based compensation)\n(2) Free cash flow = operating cash flow less capex\n(3) Includes cash, cash equivalents and investments\n(4) Calculated by dividing Cost of Automotive Sales Revenue by respective quarter’s new deliveries (ex -operating leases)Profitability 7.6% operating margin in Q3\n$1.8B GAAP operating income in Q3\n$1.9B GAAP net income in Q3\n$2.3B non -GAAP net income1 in Q3Our main objectives remained unchanged in Q3 -2023: reducing cost per \nve

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Vector database created with embeddings for the provided data


# 2. Setting custom prompt template

In [ ]:
def set_custom_prompt(system_prompt: str = config.DEFAULT_SYSTEM_PROMPT):
  """
  Prompt template for QA retrieval from vectorstore
  """
  instruction = """
  Context: {context}
  Question: {question}
  """
  B_SYS, E_SYS = "<<SYS>>\n", "\n<<SYS>>\n\n"
  B_INST, E_INST = "[INST]", "[/INST]"

  SYSTEM_PROMPT = B_SYS + system_prompt + E_SYS
  custom_prompt_template =  B_INST+ SYSTEM_PROMPT + instruction + E_INST

  prompt = PromptTemplate(
      template=custom_prompt_template,
      input_variables=['context', 'question']
      )
  return prompt

## 3. Loading LLM & Tokenizer

In [ ]:
def load_llm():
  tokenizer = AutoTokenizer.from_pretrained(config.LLM_MODEL)
  llm = AutoModelForCausalLM.from_pretrained(config.LLM_MODEL,
                                             torch_dtype=torch.float16,
                                             device_map="auto")

  llm.save_pretrained(config.LOCAL_MODEL_DIR)
  tokenizer.save_pretrained(config.LOCAL_MODEL_DIR)
  return tokenizer, llm

# 4. Retrieval QA Chain

In [ ]:
def retrieval_qa_chain(llm, prompt, db):
  qa_chain = RetrievalQA.from_chain_type(
      llm=llm,
      chain_type='stuff',
      retriever=db.as_retriever(search_kwargs={'k': 2}),
      return_source_documents=True,
      chain_type_kwargs={'prompt': prompt}
      )
  return qa_chain

# 5. Create Pipeline

In [ ]:
def create_pipeline(max_new_tokens=config.MAX_NEW_TOKENS):
  tokenizer, llm = load_llm()
  streamer = TextStreamer(tokenizer,
                          skip_prompt=True,
                          skip_special_tokens=True
                          )

  pipe = pipeline(
      "text-generation",
      model=llm,
      tokenizer=tokenizer,
      max_new_tokens=max_new_tokens,
      temperature=0,
      top_p=0.95,
      repetition_penalty=1.15,
      streamer=streamer,
  )
  return pipe

# 6. Combine above steps to build the QA bot

In [ ]:
#QA Model Function
def qa_bot():

  # fetch Embeddings from vector db
  embeddings = HuggingFaceEmbeddings(
      model_name=config.EMBEDDINGS_MODEL,
      model_kwargs={'device': config.DEVICE}
      )

  db = Chroma(persist_directory=config.DB_PATH,
              embedding_function=embeddings
              )

  #setup prompt template
  qa_prompt = set_custom_prompt(config.CUSTOM_PROMPT_TEMPLATE)

  #build pipeline
  hf_pipe = create_pipeline(max_new_tokens=config.MAX_NEW_TOKENS)

  llm = HuggingFacePipeline(pipeline=hf_pipe)

  #create qa retrieval chain
  qa = retrieval_qa_chain(llm, qa_prompt, db)
  return qa

In [ ]:
%%time
bot = qa_bot()

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

CPU times: user 10.9 s, sys: 11.5 s, total: 22.4 s
Wall time: 39.6 s


In [ ]:
# CPU times: user 10.9 s, sys: 11.5 s, total: 22.4 s
# Wall time: 39.6 s

In [ ]:
%%time
query = "what is the revenue of tesla in 2023?"
bot({"query": query})

 Thank you for providing me with this valuable context! Based on the information provided, here are my responses to your questions:
1. What was Tesla's revenue in 2023?
According to the context, Tesla's revenue in 2023 is not specified. However, the context does provide information on Tesla's revenue growth over various time periods, including 8 years ago, 4 years ago, and 1 year ago.
2. How many cars did Tesla sell in 2023?
Unfortunately, the context does not provide information on the number of cars sold by Tesla in 2023. However, it does provide information on Tesla's market share in different regions, such as North America, Europe, and Asia, during various time periods.
3. What is the installed capacity of Tesla's factories?
The context states that Tesla has an installed annual vehicle capacity of 100,000 in California, 550,000 in Model 3 / Model Y, and over 950,000 in Shanghai. It also mentions that Tesla has various next-generation platforms under development, but does not provid

{'query': 'what is the revenue of tesla in 2023?',
 'result': "  Thank you for providing me with this valuable context! Based on the information provided, here are my responses to your questions:\n1. What was Tesla's revenue in 2023?\nAccording to the context, Tesla's revenue in 2023 is not specified. However, the context does provide information on Tesla's revenue growth over various time periods, including 8 years ago, 4 years ago, and 1 year ago.\n2. How many cars did Tesla sell in 2023?\nUnfortunately, the context does not provide information on the number of cars sold by Tesla in 2023. However, it does provide information on Tesla's market share in different regions, such as North America, Europe, and Asia, during various time periods.\n3. What is the installed capacity of Tesla's factories?\nThe context states that Tesla has an installed annual vehicle capacity of 100,000 in California, 550,000 in Model 3 / Model Y, and over 950,000 in Shanghai. It also mentions that Tesla has va

In [ ]:
## Use Gradio for the front end app

In [ ]:
with gr.Blocks() as demo:

  chatbot = gr.Chatbot(label="Financial Earnings Chat Bot", height = 800)
  msg = gr.Textbox(label = "Ask Question")
  clear = gr.ClearButton([msg, chatbot])

  def respond(message, chat_history):
    bot_message = bot({"query": message})['result']
    chat_history.append((message, bot_message))
    time.sleep(0.05)
    return "", chat_history

  msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(
    share=True,
    show_api=False,
    # debug=True
)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://51dda6c412da84641b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
